## **Transformers para traducción de lenguaje**


#### **¿Por qué transformers?**

En el campo del procesamiento de lenguaje natural (NLP), a menudo se trabaja con datos secuenciales, como oraciones. Antes de los transformers, los modelos más comunes eran las redes neuronales recurrentes (RNNs) y las redes de memoria a largo plazo (LSTMs). Estos modelos procesan los datos de forma secuencial, es decir, leen una oración palabra por palabra, una tras otra. Esto los hace lentos y menos eficientes para secuencias largas. Además, las RNNs y LSTMs pueden tener dificultades para mantener la información de etapas anteriores de la secuencia, algo vital para comprender el contexto.

Los transformers introdujeron una nueva forma de procesar secuencias. En lugar de leer palabra por palabra en orden, pueden mirar toda la secuencia de una vez. Este enfoque los hace más rápidos y eficientes. También pueden comprender mejor el contexto de cada palabra en una oración, sin importar su longitud.

#### Configuración

Antes de comenzar, asegúrate de tener instaladas todas las librerías necesarias. Puedes ejecutar los siguientes comandos para instalarlas:


In [ ]:
#!pip install -U torchdata==0.5.1
#!pip install -U spacy==3.7.2
#!pip install -Uqq portalocker==2.7.0
#!pip install -qq torchtext==0.14.1
#!pip install -Uq nltk==3.8.1

#!python -m spacy download de
#!python -m spacy download en

#!pip install pdfplumber==0.9.0
#!pip install fpdf==1.7.2

!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0205EN-SkillsNetwork/Multi30K_de_en_dataloader.py'
!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/transformer.pt'
!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/input_de.pdf'

#### Importación de librerías requeridas


In [ ]:
from torchtext.datasets import multi30k, Multi30k
import torch
from typing import Iterable, List
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
from tqdm import tqdm

# También puedes usar esta sección para suprimir advertencias generadas por tu código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [ ]:
# Define símbolos especiales e índices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Asegúrate de que los tokens estén en el orden de sus índices para insertarlos correctamente en el vocabulario
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

#### **Cargador de datos (dataLoader)**

En el conjunto de datos Multi30K inglés-alemán, primero cargas los datos y descompones las oraciones en palabras o fragmentos más pequeños, llamados tokens. A partir de estos tokens creas una lista única o vocabulario. Luego, cada token se convierte en un número específico usando ese vocabulario. Dado que las oraciones pueden tener longitudes diferentes, se añade padding para igualarlas al mismo tamaño en un batch. 

Todos estos datos procesados se organizan en un `DataLoader` de PyTorch, lo que facilita su uso para entrenar redes neuronales.


In [ ]:
%run Multi30K_de_en_dataloader.py

Has configurado los dataloaders para entrenamiento y prueba.  Dado el trabajo exploratorio, usa un tamaño de batch de uno:

In [ ]:
train_dataloader, _ = get_translation_dataloaders(batch_size = 1)


Inicializa un iterador para el data loader de validación:


In [ ]:
data_itr=iter(train_dataloader)
data_itr

Para obtener ejemplos diversos, puedes iterar por múltiples muestras ya que el dataset está ordenado por longitud:



In [ ]:
for n in range(1000):
    german, english= next(data_itr)

El dataset está estructurado como secuencia-batch-feature, en lugar de batch-feature-secuencia. Para compatibilidad con las funciones auxiliares, puedes transponer los tensores:



In [ ]:
german=german.T
english=english.T

Puedes imprimir el texto convirtiendo los índices a palabras usando `index_to_german` e `index_to_english`:



In [ ]:
for n in range(10):
    german, english= next(data_itr)

    print("Muestra {}".format(n))
    print("Entrada en alemán")
    print(index_to_german(german))
    print("Objetivo en inglés")
    print(index_to_eng(english))
    print("_________\n")

Define tu dispositivo (CPU o GPU) para entrenamiento. Verificarás si hay una GPU disponible y la usarás de lo contrario, emplearás la CPU.


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

Ahora que cubriste la preparación de datos, pasemos a entender los componentes clave del transformer.


### **Conceptos importantes**


#### **Enmascaramiento (masking)**

Durante el entrenamiento, toda la secuencia está visible para el modelo y se utiliza como entrada para aprender patrones. En cambio, durante la predicción no se dispone de la parte futura de la secuencia. Para simular esta carencia de datos futuros, se emplea el enmascaramiento (masking), garantizando que el modelo aprenda a predecir sin ver los tokens siguientes reales. Esto es crucial para asegurar que ciertas posiciones no sean atendidas. 

La función `generate_square_subsequent_mask` produce una matriz triangular superior, que impide que, durante la decodificación, un token atienda a tokens futuros.

In [ ]:
def generate_square_subsequent_mask(sz,device=DEVICE):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

Por otro lado, la función `create_mask` genera tanto las máscaras de origen (source) y destino (target) como las máscaras de relleno (padding) basadas en las secuencias proporcionadas. Las máscaras de padding evitan que el modelo atienda a los tokens de relleno, simplificando la atención.

In [ ]:
def create_mask(src, tgt,device=DEVICE):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

#### **Codificación posicional (positional encoding)**

El modelo transformer no tiene conocimiento inherente del orden de los tokens en la secuencia. Para proporcionarle esta información, se añaden codificaciones posicionales a los embeddings de los tokens. Estas codificaciones siguen un patrón fijo basado en su posición en la secuencia.


In [ ]:
## Agrega información posicional a los tokens de entrada
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

#### **Embedding de tokens (token embedding)**

El embedding de tokens, también conocido como embedding de palabras o representación de palabras, convierte palabras o tokens en vectores numéricos en un espacio vectorial continuo. Cada palabra única en el corpus se representa por un vector de longitud fija cuyos valores reflejan propiedades lingüísticas, como significado, contexto o relaciones con otras palabras.

La clase `TokenEmbedding` siguiente convierte tokens numéricos en embeddings:


In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

### **Arquitectura Transformer para traducción de lenguaje**

La traducción de lenguaje con un modelo transformer se basa en una arquitectura encoder-decoder. A continuación se desglosan los componentes esenciales y el procedimiento de entrenamiento para una comprensión clara.

#### **Tokenización y codificación posicional**

Primero, el texto en el idioma origen (secuencia de entrada) se tokeniza, dividiéndolo en palabras o subpalabras. Estos tokens se convierten en representaciones numéricas y, para conservar la información de orden, se les añaden codificaciones posicionales.

#### **Procesamiento del encoder**

Estos tokens numéricos se pasan luego por el encoder, compuesto por varias capas que incluyen mecanismos de self-attention y redes feed-forward. Esta arquitectura permite al transformer procesar la secuencia completa de una vez, a diferencia de modelos secuenciales como LSTMs o GRUs.

#### **Decodificación con teacher forcing**

Durante el entrenamiento, el texto en el idioma destino (secuencia correcta de salida) también se tokeniza y convierte en tokens numéricos. El *teacher forcing* es una técnica de entrenamiento en la que el decoder recibe los tokens reales de destino como entrada. El decoder utiliza tanto la salida del encoder como los tokens generados previamente (comenzando con un token especial de inicio) para predecir el siguiente token de la secuencia.

#### **Generación de salida y cálculo de la pérdida**

El decoder genera la traducción token a token. En cada paso, predice el siguiente token de la secuencia destino. La secuencia predicha se compara con la secuencia real usando una función de pérdida, normalmente entropía cruzada, que cuantifica lo bien que coinciden las predicciones del modelo con la secuencia objetivo.

#### **Seq2SeqTransformer**

La clase `Seq2SeqTransformer` representa el núcleo del modelo transformer para traducción de lenguaje. Para entrenarla de manera efectiva, se abordarán:

* **Carga de datos:** Preparar los datos de entrenamiento (texto origen y su traducción).
* **Inicialización del modelo:** Configurar encoder, decoder, codificaciones posicionales y demás componentes.
* **Configuración del optimizador:** Elegir un optimizador (por ejemplo, Adam) y programar la tasa de aprendizaje.
* **Bucle de entrenamiento:** Iterar sobre los datos durante varias épocas, aplicando *teacher forcing*.
* **Monitoreo de la pérdida:** Registrar y, opcionalmente, graficar las pérdidas por época.

In [ ]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()

        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        outs =outs.to(DEVICE)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

### **Inferencia**

El diagrama siguiente ilustra el proceso de predicción de secuencia o inferencia. Puedes comenzar alimentando los índices de la secuencia que deseas traducir al **encoder**, representado en la sección naranja inferior izquierda. 

Los embeddings resultantes del encoder se envían al **decoder**, resaltado en verde. Además, se introduce un token de inicio (`<bos>`) al principio de la entrada del decoder, como se muestra en la base del segmento verde. 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/predict_transformers.png" alt="transformer" width="600">

La salida del decoder se mapea a un vector de tamaño igual al del vocabulario mediante una capa lineal. 

A continuación, una función softmax convierte esas puntuaciones en probabilidades. La probabilidad más alta, determinada por la función `argmax`, proporciona el índice de la palabra predicha dentro de la secuencia traducida. Este índice predicho se retroalimenta al decoder junto con la secuencia inicial, preparando el siguiente paso para determinar la próxima palabra en la traducción. 

Este proceso autorregresivo se muestra con la flecha que va desde la parte superior del decoder (verde) hasta la base.

In [ ]:
torch.manual_seed(0)

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

Comencemos con un modelo ya entrenado. Para ello, carga los pesos del transformer desde el archivo `'transformer.pt'`:

In [ ]:
transformer.load_state_dict(torch.load('transformer.pt', map_location=DEVICE, ))

Dado que el dataset está organizado por longitud de secuencia, iteremos para obtener una secuencia más larga:


In [ ]:
for n in range(100):
    src ,tgt= next(data_itr)

Muestra la secuencia de origen en alemán que deseas traducir y la secuencia objetivo en inglés que esperas que el modelo produzca:


In [ ]:
print("engish target",index_to_eng(tgt))
print("german input",index_to_german(src))

Obtén el número de tokens de la muestra en alemán:


In [ ]:
num_tokens = src.shape[0]
num_tokens

Construye una máscara para indicar qué entradas participan en el cálculo de atención. En una tarea de traducción todos los tokens de la secuencia origen están disponibles, por lo que la máscara será `False` en todas las posiciones:


In [ ]:
src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool).to(DEVICE )
src_mask[0:10]

Extrae la primera muestra del batch de secuencias a traducir. Aunque ahora parezca redundante, será útil cuando trabajes con batches más grandes:


In [ ]:
src_=src[:,0].unsqueeze(1)
print(src_.shape)
print(src.shape)


Alimenta los tokens de la secuencia al transformer junto con la máscara. El resultado, guardado en `memory`, son los embeddings producidos por el encoder:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/Transformersencoder.png" alt="trasfoemr" width="400">

La **memoria** (memory), que es la salida del encoder, encapsula la secuencia original que queremos traducir y sirve como entrada para el decoder:


In [ ]:
memory = transformer.encode(src_, src_mask)
memory.shape

Para iniciar la generación de la secuencia de salida, comienza con el símbolo de inicio  (`<bos>`):


In [ ]:
ys = torch.ones(1, 1).fill_(BOS_IDX).type(torch.long).to(DEVICE)
ys

Por convención de nombres, el término **target** se usa para denotar la predicción. En este contexto, "target" se refiere a las palabras que siguen a la predicción actual. 

Estas pueden combinarse con la secuencia de origen para realizar predicciones posteriores. Por tanto, construimos una máscara de destino donde todos los valores son `False`, indicando que no se debe ignorar ninguna posición:


In [ ]:
tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
tgt_mask


Alimentamos la salida del encoder (`memory`) y la predicción previa (por ahora solo el token de inicio) al decoder:


In [ ]:
out = transformer.decode(ys, memory, tgt_mask)
out.shape


La salida del decoder es un embedding enriquecido de la palabra predicha. A continuación, eliminamos la dimensión de batch reordenando:


In [ ]:
out = out.transpose(0, 1)
out.shape

Una vez que el decoder produce su salida, esta se pasa por la capa de salida para obtener los valores de logits sobre el vocabulario de 10 837 palabras. Más adelante solo necesitarás el último token, así que puedes usar ```out[:, -1]```:

In [ ]:
logit = transformer.generator(out[:, -1])
logit.shape

El proceso se ilustra de manera concisa en la imagen siguiente:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/decoder_start.png" alt="trasfoemr" width="600">


La palabra predicha se determina identificando el valor de logit más alto, lo cual indica la traducción más probable del modelo para la entrada en una posición específica; esta posición corresponde al índice del siguiente token.


In [ ]:
  _, next_word_index = torch.max(logit, dim=1)

In [ ]:
print("Salida en ingles:",index_to_eng(next_word_index))

Solo necesitas un entero para el índice:


In [ ]:
next_word_index=next_word_index.item()
next_word_index

Ahora, concatena la palabra recién predicha a las predicciones anteriores, permitiendo que el modelo considere toda la secuencia de palabras generadas al realizar su próxima predicción.


In [ ]:
ys = torch.cat([ys,torch.ones(1, 1).type_as(src.data).fill_(next_word_index)], dim=0)
ys

Para predecir la palabra siguiente en la traducción, actualiza la máscara de destino y utiliza el decodificador del transformer para obtener las probabilidades de las palabras. La palabra con la probabilidad más alta es entonces seleccionada como predicción. 

Ten en cuenta que la salida del codificador contiene toda la información que necesitas.


In [ ]:
# Actualiza la máscara de destino para la longitud de la secuencia actual.
tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
tgt_mask

In [ ]:
# Decodifica la secuencia actual utilizando el transformer y recupera la salida
out = transformer.decode(ys, memory, tgt_mask)
out = out.transpose(0, 1)
out.shape

In [ ]:
out[:, -1].shape

In [ ]:
# Obtiene las probabilidades de palabras para la última palabra predicha.
prob = transformer.generator(out[:, -1])
# Encuentra el índice de palabra con la probabilidad más alta.
_, next_word_index = torch.max(prob, dim=1)
# Imprime la palabra en inglés predicha.
print("Salida en inglés:", index_to_eng(next_word_index))
# Convierte el valor del tensor a un escalar de Python.
next_word_index = next_word_index.item()


Ahora, actualiza la predicción.


In [ ]:
ys = torch.cat([ys,torch.ones(1, 1).type_as(src.data).fill_(next_word_index)], dim=0)
print("Salida en inglés",index_to_eng(ys))

El proceso puede resumirse de la siguiente manera: comenzando con la salida inicial del codificador y el token `<BOS>`, la salida del decodificador se retroalimenta en el decodificador hasta que la secuencia traducida se decodifica por completo. 

Este ciclo continúa hasta que la longitud de la nueva secuencia traducida coincida con la de la secuencia original. Como se muestra en la siguiente imagen, la función `greedy_decode` también está incluida.
    
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/decoder.png" alt="trasfoemr" width="500">


In [ ]:
def greedy_decode(modelo, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = modelo.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = modelo.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = modelo.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

Se recupera los índices para el idioma alemán y genera la máscara correspondiente:

In [ ]:
src
src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool).to(DEVICE )

Establece un valor razonable para la longitud máxima de la secuencia objetivo:


In [ ]:
max_len=src.shape[0]+5
max_len

Aplica la función `greedy_decode` a los datos:


In [ ]:
ys=greedy_decode(transformer, src, src_mask, max_len, start_symbol=BOS_IDX)
print("Inglés ",index_to_eng(ys))


Observa que funciona, pero no es exactamente lo mismo. 

In [ ]:
print("Inglés ",index_to_eng(tgt))

### **Entrenamiento vs. inferencia en traducción automática neuronal**

Durante la fase de inferencia, cuando el modelo se despliega para tareas reales de traducción, el decodificador genera la secuencia sin tener acceso a la secuencia objetivo esperada. En su lugar, basa sus predicciones en la salida del codificador y en los tokens que ha generado hasta el momento. 

El proceso es autorregresivo, con el decodificador prediciendo continuamente el siguiente token hasta que produce un token de fin de secuencia, indicando que la traducción está completa.

La principal diferencia entre las etapas de entrenamiento e inferencia radica en las entradas al decodificador. Durante el entrenamiento, el decodificador se beneficia de la exposición de los ground truth, recibiendo los tokens exactos de la secuencia objetivo de manera incremental a través de una técnica conocida como "teacher forcing". 

Este enfoque contrasta marcadamente con algunas otras arquitecturas de redes neuronales que confían en las predicciones anteriores de la propia red como entradas durante el entrenamiento. Una vez finalizado el entrenamiento, los conjuntos de datos utilizados se asemejan a los empleados en modelos de redes neuronales más convencionales, proporcionando una base familiar para la comparación y evaluación.

Primero, importa la pérdida `CrossEntropyLoss` y crea un objeto de pérdida de entropía cruzada. La pérdida **no** se calculará cuando el token con índice `PAD_IDX` sea una entrada.

In [ ]:
from torch.nn import CrossEntropyLoss

loss_fn = CrossEntropyLoss(ignore_index=PAD_IDX)

Elimina la última muestra del objetivo:


In [ ]:
tgt_input = tgt[:-1, :]
print(index_to_eng(tgt_input))
print(index_to_eng(tgt))

Crea las máscaras requeridas:

In [ ]:
src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
print(f"Forma de src_mask: {src_mask.shape}")
print(f"Forma de tgt_mask: {tgt_mask.shape}")
print(f"Forma de src_padding_mask: {src_padding_mask.shape}")
print(f"Forma de tgt_padding_mask: {tgt_padding_mask.shape}")

In [ ]:
src_padding_mask

En la máscara de destino, cada columna subsecuente revela incrementamente más tokens al introducir valores de infinito negativo, desbloqueándolos. Puedes mostrar la máscara de destino para visualizar el progreso o identificar específicamente qué tokens están siendo enmascarados en cada paso.


In [ ]:
print(tgt_mask)
[index_to_eng( tgt_input[t==0])  for t in tgt_mask] #index_to_eng(tgt_input))

Cuando llamas a `modelo(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)`, se invoca el método forward de la clase `Seq2SeqTransformer`. 

Este proceso genera *logits* para la secuencia objetivo, los cuales pueden luego convertirse en tokens reales tomando la predicción de mayor probabilidad en cada paso de la secuencia.


#### **Función de pérdida**


Profundicemos en cómo puedes calcular la pérdida a partir de tu `src` y `tgt_input`:

In [ ]:
logits = transformer(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

print("Formato de la salida ",logits.shape)
print("Formato del objetivo",tgt_input.shape)
print("Formato de la fuente ",src.shape)

Durante la fase de entrenamiento, un aspecto intrigante y sofisticado del proceso es la doble funcionalidad del objetivo. Actúa simultáneamente como entrada para el decodificador del transformer y como estándar contra el cual se mide la exactitud de la predicción. 

Para mayor claridad en las discusiones, nos referimos al objetivo usado como entrada para el decodificador como "entrada al decodificador". 

Por otro lado, el "ground truth para la predicción", que es la secuencia objetivo desplazada a la derecha, dado que es un modelo autorregresivo, se conocerá simplemente como "objetivo de salida" en lo sucesivo.


Ground truth para la predicción se desplaza simplemente a la derecha y se llama `tgt_out`. 

Puedes imprimir los tokens:


In [ ]:
tgt_out = tgt[1:, :]
print(tgt_out.shape)
[index_to_eng(t)  for t in tgt_out]

Los índices de los tokens representan las clases que deseas predecir. Al aplanar el tensor, cada índice se convierte en una muestra distinta, sirviendo como objetivo para la pérdida de entropía cruzada.


In [ ]:
tgt_out_flattened = tgt_out.reshape(-1)
print(tgt_out_flattened.shape)
tgt_out_flattened

En este modelo autorregresivo, muestra los tokens de entrada del objetivo después de aplicar la máscara. Junto a ellos, puedes mostrar la salida objetivo, ilustrando cómo el modelo predice hábilmente valores pasados basándose en los presentes. Esta visualización clara resalta la capacidad del modelo para usar la información actual e inferir lo que ha precedido, una característica clave de su naturaleza autorregresiva.


In [ ]:
["Entrada: {} objetivo: {}".format(index_to_eng( tgt_input[m==0]),index_to_eng( t))  for m,t in zip(tgt_mask,tgt_out)] 

Ahora, calcula la pérdida ya que la salida del decodificador del transformers se proporciona como entrada a la función de pérdida de entropía cruzada junto con los valores de la secuencia objetivo. 

Dado que la salida del transformer tiene las dimensiones longitud de secuencia, tamaño de lote y características (`vocab_size`), es necesario remodelar esta salida para alinearla con el formato de entrada estándar requerido por la función de pérdida de entropía cruzada. 

Este paso asegura que la pérdida se calcule correctamente, comparando la secuencia predicha con los ground truths en cada paso de tiempo en todo el lote usando el método reshape.


In [ ]:
loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
print(loss)

#### **El cálculo de la función pérdida**

Eso es todo para el cálculo de la pérdida, pero si tienes curiosidad sobre cómo se calcula la pérdida aquí, esto es lo que sucede internamente al calcular la pérdida de entropía cruzada. 

Primero, verifica la forma de los tensores antes y después del remodelado:


In [ ]:
# logits.reshape(-1, logits.shape[-1]) transforma el tensor logits en un tensor 2D con forma [longitud_de_secuencia * tamaño_de_lote, tamaño_del_vocabulario]. Este cambio de forma se realiza para alinear tanto los logits predichos como las salidas objetivo para el cálculo de la pérdida.
print("La forma de logits es:", logits.shape)
logits_flattened = logits.reshape(-1, logits.shape[-1])
print("La forma de logit_flats es:", logits_flattened.shape)

# tgt_out.reshape(-1) convierte el tensor tgt_out en un tensor 1D al aplanarlo a lo largo de las dimensiones de secuencia y tamaño de lote. Esto se hace para alinearlo con los logits transformados.
print("La forma de tgt_outs es:", tgt_out.shape)
tgt_out_flattened = tgt_out.reshape(-1)
print("La forma de tgt_out_flats es:", tgt_out_flattened.shape)


Dentro de la función de pérdida, los logits se transformarán en probabilidades entre `[0,1]` que suman 1:

In [ ]:
# Aplicando la función de pérdida de la entropia cruzada
probs = torch.nn.functional.softmax(logits_flattened, dim=1)
probs[1].sum()

Verifica las probabilidades para algunos tokens aleatorios:


In [ ]:
### Tu respuesta

In [ ]:
for i in range (5):
    # using argmax, you can retrieve the index of the token that is predicted with the highest probaility
    print("Id del token predicho:",probs[i].argmax().item(), "probabilidad predicha:",probs[i].max().item())
    # you can also check the actual token from the tgt_out_flat
    print("Id del token actual:",tgt_out_flattened[i].item(), "probabilidad predicha:", probs[i,tgt_out_flattened[i]].item(),"\n")

Se puede observar que para muchos tokens el modelo está haciendo un buen trabajo al predecir el token, mientras que para algunos de los tokens el modelo no asigna una alta probabilidad al token real que se debe predecir. La diferencia entre la probabilidad predicha para dichos tokens es la razón por la que la pérdida no sumaría 0.

Ahora, puedes proceder a calcular la diferencia entre la probabilidad del token real (1) y las probabilidades predichas para cada token:

In [ ]:
neg_log_likelihood = torch.nn.functional.nll_loss(probs, tgt_out_flattened)
# Obteniendo el valor de perdida 
loss = neg_log_likelihood

# Imprime el valor total de perdida
print("Función de pérdida:", loss.item())

#### **Evaluación**

Siguiendo los procedimientos mencionados anteriormente, puedes desarrollar una función capaz de hacer predicciones y, posteriormente, calcular la pérdida correspondiente en los datos de validación. 


In [ ]:
def evaluate(modelo):
    modelo.eval()
    losses = 0

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = modelo(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

#### **Entrenando el modelo**

Incorporando los pasos descritos anteriormente, procedemos a entrenar el modelo. Además de estos procedimientos específicos, el proceso de entrenamiento general se ajusta a los métodos convencionales empleados en el entrenamiento de redes neuronales. 

Ahora, escribimos una función para entrenar el modelo.


In [ ]:
def train_epoch(modelo, optimizer, train_dataloader):
    modelo.train()
    losses = 0

    # Envuelve train_dataloader con tqdm para registrar el progreso
    train_iterator = tqdm(train_dataloader, desc="Training", leave=False)

    for src, tgt in train_iterator:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        src_mask = src_mask.to(DEVICE)
        tgt_mask = tgt_mask.to(DEVICE)
        src_padding_mask = src_padding_mask.to(DEVICE)
        tgt_padding_mask = tgt_padding_mask.to(DEVICE)

        logits = modelo(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
        logits = logits.to(DEVICE)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

        # Actualiza la barra de progreso de tqdm con la pérdida actual
        train_iterator.set_postfix(loss=loss.item())

    return losses / len(list(train_dataloader))

La configuración para el modelo de traducción incluye un tamaño de vocabulario de origen y destino determinado por los idiomas del conjunto de datos, un tamaño de embedding de 512, 8 cabecera de atención, una dimensión oculta para la red feed-forward de 512 y un tamaño de lote de 128. 

El modelo está estructurado con tres capas tanto en el encoder como en el decoder.



In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

Crea un loader de entrenamiento con un tamaño de lote de 128.

In [ ]:
train_dataloader, val_dataloader = get_translation_dataloaders(batch_size = BATCH_SIZE)

Crea un modelo de transformer:


In [ ]:
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
transformer = transformer.to(DEVICE)

Inicializa los pesos del modelo transformer.


Prepara el optimizador Adam.


In [ ]:
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

Inicializa las listas de pérdidas de entrenamiento y validación.

In [ ]:
TrainLoss=[]
ValLoss=[]

Entrena el modelo durante 10 épocas usando las funciones anteriores.

>Ten en cuenta que entrenar el modelo usando CPUs puede ser un proceso que lleva mucho tiempo.
>Si no tienes acceso a  GPUs, puede pasar directamente a "cargar el modelo guardado" y proceder a cargar el modelo preentrenado usando el código proporcionado. 

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 10

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, train_dataloader)
    TrainLoss.append(train_loss)
    end_time = timer()
    val_loss = evaluate(transformer)
    ValLoss.append(val_loss)
    print((f"Epoca: {epoch}, Pérdida de entrenamiento: {train_loss:.3f}, Pérdida de validación: {val_loss:.3f}, "f"Tiempo de epocas = {(end_time - start_time):.3f}s"))
torch.save(transformer.state_dict(), 'transformer_de_to_en_model.pt')

Dibujamos la pérdida de los datos de entrenamiento y validación

In [ ]:
epochs = range(1, len(TrainLoss) + 1)

plt.figure(figsize=(10, 5))
plt.plot(epochs, TrainLoss, 'r', label='Pérdida de entrenamiento')
plt.plot(epochs,ValLoss, 'b', label='Pérdida de validación')
plt.title('Pérdida de entrenamiento y validación')
plt.xlabel('Epocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

#### **Carga del modelo guardado**

Si quieres omitir el entrenamiento y cargar el modelo preentrenado que se proporciona, descomenta la siguiente celda:

In [ ]:
#!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/transformer_de_to_en_model.pt'
#transformer.load_state_dict(torch.load('transformer_de_to_en_model.pt',map_location=torch.device('cpu')))

### **Traducción y evaluación**

Usando la función `greedy_decode` que definiste anteriormente, puedes crear una función traductor que genere la traducción al inglés de un texto alemán de entrada.


In [ ]:
# Traduce la frase de entrada al idioma objetivo
def translate(modelo: torch.nn.Module, src_sentence: str):
    modelo.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        modelo,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

Ahora, veamos algunas traducciones de muestra:


In [ ]:
for n in range(5):
    german, english = next(data_itr)

    print("Oración en alemán:", index_to_german(german).replace("<bos>", "").replace("<eos>", ""))
    print("Traducción al inglés:", index_to_eng(english).replace("<bos>", "").replace("<eos>", ""))
    print("Traducción del modelo:", translate(transformer, index_to_german(german)))
    print("_________\n")

### **Evaluación con la puntuación BLEU**

Para evaluar las traducciones generadas, se introduce una función `calculate_bleu_score`. Esta calcula la puntuación BLEU, una métrica común para la calidad de la traducción automática, comparando la traducción generada con traducciones de referencia. La puntuación BLEU ofrece una medida cuantitativa de la precisión de la traducción.

El código también incluye un ejemplo de cómo calcular la puntuación BLEU para una traducción generada.


In [ ]:
def calculate_bleu_score(generated_translation, reference_translations):
    # convierte las traducciones generadas y las traducciones de referencia al formato esperado por sentence_bleu
    references = [reference.split() for reference in reference_translations]
    hypothesis = generated_translation.split()

    # calcula la puntuación BLEU
    bleu_score = sentence_bleu(references, hypothesis)

    return bleu_score

In [ ]:
generated_translation = translate(transformer,"Ein brauner Hund spielt im Schnee .")

reference_translations = [
    "A brown dog is playing in the snow .",
    "A brown dog plays in the snow .",
    "A brown dog is frolicking in the snow .",
    "In the snow, a brown dog is playing ."

]

bleu_score = calculate_bleu_score(generated_translation, reference_translations)
print(" Puntuación de BLEU:", bleu_score, "for",generated_translation)

#### **Ejercicio: Traducir un documento**

En este ejercicio, implementarás una función que traduzca un PDF en alemán a inglés. Para lograrlo, utilizarás el mismo modelo de transformer de secuencia a secuencia discutido previamente y realizarás las modificaciones necesarias.

1. **Definir la función de traducción**
   Crea una función llamada `translate_pdf` que reciba los siguientes parámetros:

   * `input_file`: La ruta al archivo PDF de entrada que se va a traducir.
   * `translator_model`: Un modelo o función que manejará la traducción del texto.
   * `output_file`: La ruta donde se guardará el PDF traducido.

2. **Leer y traducir el PDF**
   Utiliza `pdfplumber` para abrir y leer el texto de cada página del PDF de entrada. Traduce el texto extraído usando el `translator_model`.

3. **Formatear y escribir el texto traducido en un nuevo PDF**

   * Usa `textwrap` para ajustar el texto traducido de manera que se adapte al ancho de página A4.
   * Crea un nuevo PDF con `FPDF` y añade el texto traducido ajustado.
   * Guarda el nuevo PDF con el texto traducido en `output_file`.


In [ ]:
import pdfplumber
import textwrap
from fpdf import FPDF

def translate_pdf(input_file, translator_model,output_file):
    #Completa la función

Aquí tienes un documento en alemán para que lo conviertas:

In [ ]:
!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/input_de.pdf'

Ahora llama a translate_pdf para el archivo alemán como entrada a la función y verifica el archivo de salida para el archivo traducido.

In [ ]:
input_file_path =
output_file = 'output_en.pdf'
# Llama a translate_pdf() definido anteriormente para el archivo de entrada
print("El archivo PDF traducido se guarda como:", output_file)


In [ ]:
## Tus respuestas